# 1. Introduction and Business Problem

## Context
We want to open a new restaurant in city in Italy, Riccione. While this town is quite small (35K inhabitants), it is a vary famous holiday location during summer among both younger people, looking to have fun, and families, looking for some peace and quiet at the beach.

## Town Morphology
Riccione is traversed by two main streets, where the highes number of places are situated. Those streets, named Viale Dante and Viale Ceccarini, cross themselfs at the end of each other (forming a big T) while Viale Dante is parallel to the sea. On top of that, since we are in a marine location, the highest number of places is situated near the sea and the harbor. On the other hand the outback is still rich in places, while having a lower density.

## People Abitudes
If the town is so little, then each place is so close to every other so that people always choose the best place to eat, right? Wrong! In summer the town is practically impossible to navigate using a car, either due to heavy traffic load or lack of physical space where to park (belive it or not). This means that people tend to visit places that are quite near the Hotel where they stay. This concept is very important to understand why we choose one approach and why the result is satisfying.

## Goal
In the end the goal is to find one or more small areas where to open a restaurant based on the quality of potential neighborhood competitors.

# 2. Description of the data set
## Source

The data set will be provided via foursquare api, and will be such that, given a place location, we have a value that represents how much the place is appreciated by people.

## Examples

Here we have a bare example of what data we need, related to a place, to perform our analysis

| **Name**                 | **Latitude**  | **Longitude** | **Rating** |
|----------------------|-----------|-----------|--------|
| Locanda della luna   | 44.011014 | 12.632975 | 8.1    |
| La Fattoria del mare | 44.016893 | 12.641178 | 7.9    |

## How to retrieve the data

How do we retrieve the data, considering that the town is not circular (since it is near the sea)? How do we capture exactly the bounds of the town without considering places from nearby cities? 
The idea behind the process is to bound our research to a certain number of points on a map (latitude and longitude) so that providing them a certain radius we can cover the entire city shape and no more.

# 3. Methodology

It is trivial to see that the problem falls into the clustering category. In fact our goal is to find some areas (thus unknown) where there is a concentration of points such that they represent mediocre restaurants, thus creating our business opportunity. We want to cluster all the places in the town, including those that are not near the sea.

## Data Exploration

The best way to explore location data is to plot it. Looking at the points of the map (you can execute the code notebook and generate it or look at the presentation) we can see that points are not exactly evenly distributed across the city. In fact, as noted in the introduction, the city is mainly developed near the sea. On top of that we can see that the api returns places that are not of the chosen category (food); we will need to eliminate those records. In the end we eliminate all records that do not have a rating and duplicate records, since we run our query multiple times to cover the entire city.

As regards the rating, we consider a place to be very good if it has a rating score above 8, while we consider it a mediocre place if it has a  rating score below 7 (according to Foursquare this means "discordant opinions"). Everything that falls in 7-8 bucket is thus considered "average" quality.

## Data Manipulation

At this point our data set should look like this:

| **Name**                 | **Latitude**  | **Longitude** | **Rating** |
|----------------------|-----------|-----------|--------|
| Locanda della luna   | 44.011014 | 12.632975 | 8.1    |
| La Fattoria del mare | 44.016893 | 12.641178 | 7.9    |

Lat long coordinates have a problem that they are two features that represent a three dimensional space. This means that the long coordinate goes all around and potentially the two most extreme values are actually very close together. While this is not the case (we are analyzing a small town) it is always safe to map latitude and longitude to the corresponding **3-dimensional space**:

x = cos(lat) * cos(lon)

y = cos(lat) * sin(lon)

z = sin(lat) 

On top of that, since we have to take into account the Rating, overall we are clustering in a 4-dimensional space of features with different range span (Rating has a much higher value span than our 3-dimension location space) and we need to normalize it. In this case a simple MinMax approach will be enough to equal the variability of our 4 dimensions. 

## Choosing the right algorithm

We have two main options with which we can procede in our clustering problem:

* Classic clustering algorithm, such as KMeans
* Density based clustering, such as DBSCAN

For this particular problem, since we know that the town has various zones with very different densities and we want all of our points inside a cluster, the best approach would be using KMeans.


# 4. Results

## A first look

After running our KMeans multiple times and plotting the "Elbow", we see that going further 7 clusters does not improve significantly our information gain (or does not decrease significantly our loss).

Taking a closer look at those 7 clusters we see that:

* 5 of them are near the sea, all around Viale Dante
* The other 2 of them are divided in the outback
* 2 Clusters (one near the sea and one in the outback) have a suspiciously high number of places with rating below 7
* 2 Clusters near the harbor overlap each other spatially (looks like one of them has a very high number of "good places" compared to the other)

# A closer look

Looking closer at our clusters we can obtain the average rank for each of our group:

| Cluster label | Cluster Average Rank |
|---------------|----------------------|
|       1       |       7.327273       |
|       2       |       6.821429       |
|       3       |       7.350000       |
|       4       |       7.309091       |
|       5       |       8.318182       |
|       6       |       6.729630       |

The data confirms our hypothesis: cluster 2 and cluster 6 have an average rank below our "mediocre" threshold, while cluster 5 (the one near the harbor) has a very high average "quality" compared to all other clusters.

# Where to place our restaurant?

In the end a good point to start is placing the Centroid of cluster 2 and cluster 6 and start looking for all the possible places that can be rent to build our new awesome restaurant! In general, the area covered by those two clusters are good starting points where to open our new activity.

# 5. Discussion
## Enphasis on choosing the right algorithm

While working with location data it is very common to instantly think to Density Based Clustering. This is due to all the good properties that algorithms like DBSCAN have, like high outlier tolerance, indipendence to K (number of clusters) and various cluster forms.


On the other hand in our case we needed to consider ALL points of the dataset, even those who are "far away" from the others. On top of that it is very common to forget that Density based Algorithms work with the concept of **Fixed density**, and that they are not suitable if the dataset shows different densities among clusters.

In the end choosing the right algorithm makes a big difference in giving the best result to a certain problem.

## Data range and normalization

We know that the scaling procedure used in our methodology is MinMax. This forces our data to express a value between 0 and 1.
While looking at the range of our data you might notice that our spatial coordinates (x,y,z) are already in the 0 - 1 range, so why we should normalize them too?

This is because, having a closer look, our spatial coordinates (x,y,z) are theoretically in the 0-1 range, but in our case their variability is **incredibly small** due to the fact that they represent points very close to each other on the earth surface. If you normalize only the Rating value, while clustering you will see that the rating itself is the only discriminant when choosing which cluster a point belongs to. This is because, while having the same range, the two values express a different **variability**. Always think twice when scaling features!

# 6. Conclusion

In the end our experiment was a success, and we correctly identified two zones where a potential investor could open a restaurant. 